In [2]:
import requests
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import wikipedia
from PIL import Image
from io import BytesIO


c:\Users\andre.de.souza.reis\Anaconda\envs\streamlit\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [42]:
data = requests.request("GET", 'http://ergast.com/api/f1/drivers?limit=10000').text
pilots = pd.read_xml(
    data,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Driver'
    )
pilots = pilots.assign(Name = pilots[['GivenName', 'FamilyName']].apply(' '.join, axis = 1)).drop(['GivenName', 'FamilyName'], axis = 1)
# simmilar = process.extractBests("Lewis", pilots['Name'].to_list(), limit = 10, score_cutoff=80)

# a = []
# for i in simmilar:
#     a.append(i[0])
# tuple(a)

In [43]:
pilots

,driverId,url,DateOfBirth,Nationality,code,PermanentNumber,Name
0,abate,http://en.wikipedia.org/wiki/Carlo_Mario_Abate,1932-07-10,Italian,None,NaN,Carlo Abate
1,abecassis,http://en.wikipedia.org/wiki/George_Abecassis,1913-03-21,British,None,NaN,George Abecassis
2,acheson,http://en.wikipedia.org/wiki/Kenny_Acheson,1957-11-27,British,None,NaN,Kenny Acheson
3,adams,http://en.wikipedia.org/wiki/Philippe_Adams,1969-11-19,Belgian,None,NaN,Philippe Adams
4,ader,http://en.wikipedia.org/wiki/Walt_Ader,1913-12-15,American,None,NaN,Walt Ader
...,...,...,...,...,...,...,...
849,zapico,http://en.wikipedia.org/wiki/Emilio_Zapico,1944-05-27,Spanish,None,NaN,Emilio Zapico
850,zhou,http://en.wikipedia.org/wiki/Guanyu_Zhou,1999-05-30,Chinese,ZHO,24.0,Guanyu Zhou
851,zonta,http://en.wikipedia.org/wiki/Ricardo_Zonta,1976-03-23,Brazilian,ZON,NaN,Ricardo Zonta
852,zorzi,http://en.wikipedia.org/wiki/Renzo_Zorzi,1946-12-12,Italian,None,NaN,Renzo Zorzi


In [ ]:
# text = requests.request("GET", 'http://en.wikipedia.org/wiki/Lewis_Hamilton').text

# sample = "//upload.wikimedia.org/wikipedia/commons/thumb"

# start = text.find(sample)
# end = text[start:].find('"')

# 'https:' + text[start:(start + end)]

In [ ]:
# text = requests.request("GET", pilots[pilots['Name'] == 'Lewis Hamilton']['url'].values[0]).text

# sample = "//upload.wikimedia.org/wikipedia/commons/thumb"

# start = text.find(sample)
# end = text[start:].find('"')

# 'https:' + text[start:(start + end)]

In [ ]:
# url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Lewis_Hamilton_2016_Malaysia_2.jpg/640px-Lewis_Hamilton_2016_Malaysia_2.jpg'

# response = requests.get(url)

# Image.open(BytesIO(response.content))

In [5]:
data = requests.request("GET", 'http://ergast.com/api/f1/drivers/alonso').text


'<?xml version="1.0" encoding="utf-8"?>\n<?xml-stylesheet type="text/xsl" href="/schemas/mrd-1.5.xsl"?>\n<MRData xmlns="http://ergast.com/mrd/1.5" series="f1" url="http://ergast.com/api/f1/drivers/alonso" limit="30" offset="0" total="1">\n\t<DriverTable driverId="alonso">\n\t\t<Driver driverId="alonso" code="ALO" url="http://en.wikipedia.org/wiki/Fernando_Alonso">\n\t\t\t<PermanentNumber>14</PermanentNumber>\n\t\t\t<GivenName>Fernando</GivenName>\n\t\t\t<FamilyName>Alonso</FamilyName>\n\t\t\t<DateOfBirth>1981-07-29</DateOfBirth>\n\t\t\t<Nationality>Spanish</Nationality>\n\t\t</Driver>\n\t</DriverTable>\n</MRData>\n'

In [10]:
pilots = pd.read_xml(
    data,
    namespaces = {'doc': "http://ergast.com/mrd/1.5"},
    xpath = './/doc:Driver'
    )

str